# SlideRule Voila Demo

#### SlideRule is an on-demand science data processing service that runs in the cloud and responds to REST API calls to process and return science results.

This web page demonstrates a simple use of SlideRule to return elevations within a small region of interest using processing parameters specified at the time of the request.  The web page is implemented inside a Jupyter Notebook using SlideRule's Python client, and is statically served using Voila.  For more information on SlideRule, and how to install and use it for your own analysis applications, please see our website at [icesat2sliderule.org](http://icesat2sliderule.org).

| Step 1 | Step 2 | Step 3 | Step 4 | Step 5 |
|--------|--------|--------|--------|--------|
| Select Region of Interest | Choose Processing Parameters and Run SlideRule | Explore Data Points and Refresh Plot | Plot Photon Cloud of Ground Track | Build Your Own Notebook |
| Use the map to draw a polygon or bounding box around your region of interest. | Select the processing parameters to use to calculate gridded elevations for the region of interest created above. Then click the "Run SlideRule!" button to initiate a https://github.com/ICESat2-SlideRule/sliderule-python/blob/main/examples/api_widgets_demo.ipynb request to SlideRule. Once all granules have been processed, the above map will be updated with a scatter plot of the different elevations. | The results are returned as a GeoDataFrame with multiple columns. Choose a field in the returned results to plot, and then click "Refresh Plot" to see the map updated with values from that column. | Enter the reference ground track, the cycle, and the individual ground track in the input boxes below. If you click on an individual elevation in the map, it will automatically populate these inputs with the correct values. | Check out our [API widgets demo](https://github.com/ICESat2-SlideRule/sliderule-python/blob/main/examples/api_widgets_demo.ipynb) for a guided walkthrough of how SlideRule works along with code you can use to start your own notebook. |

## Gridded Elevations ([atl06p](http://icesat2sliderule.org/rtd/api_reference/icesat2.html#atl06p))

In [ ]:
# load the necessary packages
from sliderule import icesat2, ipysliderule, io
import ipywidgets as widgets
import geopandas
import logging
import warnings
import time
# atl03 plotting imports
import numpy as np
import matplotlib.lines
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# turn off warnings for demo
warnings.filterwarnings('ignore')

In [ ]:
# create global variables
url_textbox = None
atl06_rsps = None
parms = None
results = []
SRwidgets = ipysliderule.widgets()
points_dropdown = None
run_button = widgets.Button(description="Run SlideRule!")
run_output = widgets.Output()
refresh_button = widgets.Button(description="Refresh Plot")
refresh_output = widgets.Output()

In [ ]:
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet('Global', layer_control=False)
display(m.map)
display(run_output)

In [ ]:
%matplotlib inline
# callback for handling results
def demoCB(resource, result, index, total):
    global parms, results
    print(f'SlideRule processing request... [{index}/{total}] {resource} returned {len(result)} elevations\r', end="")
    if len(result) > 0:
        results.append(result)

# build and transmit requests to SlideRule
def runSlideRule():
    global url_textbox, parms, results
    
    # set the url for the sliderule service
    if url_textbox.value == 'local':
        url = 'host.docker.internal'
    else:
        url = url_textbox.value
    icesat2.init(url, loglevel=logging.WARNING, max_resources=1000)

    # sliderule asset and data release
    asset = SRwidgets.asset.value

    # build sliderule parameters using latest values from widget
    parms = {
        # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
        "srt": SRwidgets.surface_type.index,
        # length of ATL06-SR segment in meters
        "len": SRwidgets.length.value,
        # step distance for successive ATL06-SR segments in meters
        "res": SRwidgets.step.value,
        # confidence level for PE selection
        "cnf": SRwidgets.confidence.value,
        # ATL08 land surface classifications
        "atl08_class": list(SRwidgets.land_class.value),
        # maximum iterations, not including initial least-squares-fit selection
        "maxi": SRwidgets.iteration.value,
        # minimum along track spread
        "ats": SRwidgets.spread.value,
        # minimum PE count
        "cnt": SRwidgets.count.value,
        # minimum height of PE window in meters
        "H_min_win": SRwidgets.window.value,
        # maximum robust dispersion in meters
        "sigma_r_max": SRwidgets.sigma.value
    }

    # clear existing geodataframe
    results = []
    rsps = icesat2.__emptyframe()

    # for each region of interest
    for poly in m.regions:
        # add polygon from map to sliderule parameters
        parms["poly"] = poly 
        # make the request to the SlideRule (ATL06-SR) endpoint
        # and pass it the request parameters to request ATL06 Data
        icesat2.atl06p(parms, asset, callback=demoCB)

    if len(results) > 0:
        rsps = geopandas.pd.concat(results)
    
    return rsps

# run sliderule action
def on_run_clicked(b):
    global atl06_rsps, points_dropdown
    with run_output:
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        atl06_rsps = runSlideRule()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {atl06_rsps.shape[0]} elevations                                   ')
        if atl06_rsps.shape[0] > 0:
            max_plot_points = 10000
            if points_dropdown.value == "100K":
                max_plot_points = 100000
            elif points_dropdown.value == "unlimited":
                max_plot_points = 1000000000
            m.GeoData(atl06_rsps, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap, max_plot_points=max_plot_points)

# refresh action
def on_refresh_clicked(b):
    global atl06_rsps
    with refresh_output:
        if atl06_rsps.shape[0] > 0:
            max_plot_points = 10000
            if points_dropdown.value == "100K":
                max_plot_points = 100000
            elif points_dropdown.value == "unlimited":
                max_plot_points = 1000000000
            m.GeoData(atl06_rsps, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap, max_plot_points=max_plot_points)
        
# link buttons
run_button.on_click(on_run_clicked)
refresh_button.on_click(on_refresh_clicked)

In [ ]:
# url input text box
url_textbox = widgets.Text(
    value='local',
    placeholder='Input box for SlideRule url',
    description='URL:',
    disabled=False
)
display(url_textbox)

# points to plot drop down
points_dropdown = widgets.Dropdown(
    options = ["10K", "100K", "unlimited"],
    value = "10K",
    description = "Pts to Draw",
    disabled = False,
)

# display widgets for setting SlideRule parameters
display(widgets.VBox([
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma,
    SRwidgets.variable,
    SRwidgets.cmap,
    points_dropdown,
    SRwidgets.reverse
]))

# display buttons
display(run_button)
display(refresh_button, refresh_output)

In [ ]:
# globals for atl03 processing
atl03_rsps = None
elev_dropdown = None
pc_button = widgets.Button(description="Plot Photon Cloud")
pc_output = widgets.Output()

# track lookup tables
gt2str = {icesat2.GT1L: 'gt1l', icesat2.GT1R: 'gt1r', icesat2.GT2L: 'gt2l', icesat2.GT2R: 'gt2r', icesat2.GT3L: 'gt3l', icesat2.GT3R: 'gt3r'}
str2gt = {'gt1l': icesat2.GT1L, 'gt1r': icesat2.GT1R, 'gt2l': icesat2.GT2L, 'gt2r': icesat2.GT2R, 'gt3l': icesat2.GT3L, 'gt3r': icesat2.GT3R}
gtlookup = {icesat2.GT1L: 1, icesat2.GT1R: 1, icesat2.GT2L: 2, icesat2.GT2R: 2, icesat2.GT3L: 3, icesat2.GT3R: 3}
pairlookup = {icesat2.GT1L: 0, icesat2.GT1R: 1, icesat2.GT2L: 0, icesat2.GT2R: 1, icesat2.GT3L: 0, icesat2.GT3R: 1}

## Photon Cloud ([atl03sp](http://icesat2sliderule.org/rtd/api_reference/icesat2.html#atl03spElevations))

In [ ]:
%matplotlib widget
# ATL03 Subsetter
def runATL03Subsetter():
    global url_textbox, parms
    
    # set the url for the sliderule service
    if url_textbox.value == 'local':
        url = 'host.docker.internal'
    else:
        url = url_textbox.value
    icesat2.init(url, loglevel=logging.WARNING)

    # sliderule asset and data release
    asset = SRwidgets.asset.value

    # build sliderule parameters using latest values from widget
    parms = {
        # processing parameters
        "srt": SRwidgets.surface_type.index,
        "len": SRwidgets.length.value,
        "res": SRwidgets.step.value,

        # classification and checks
        "pass_invalid": True, # still return photon segments that fail checks
        "cnf": -2, # all photons
        "atl08_class": ["atl08_noise", "atl08_ground", "atl08_canopy", "atl08_top_of_canopy", "atl08_unclassified"],
        "yapc": {"score": 0}, # all photons
        "ats": SRwidgets.spread.value,
        "cnt": SRwidgets.count.value,
        
        # region of interest
        "poly": m.regions[0],
        
        # track selection
        "rgt": int(SRwidgets.rgt.value),
        "cycle": int(SRwidgets.cycle.value),
        "track": gtlookup[str2gt[SRwidgets.ground_track.value]]
    }

    # make call to sliderule
    rsps = icesat2.atl03sp(parms, asset)
    
    # return geodataframe
    return rsps

# photon_cloud action
def on_pc_clicked(b):
    global atl03_rsps, atl06_rsps, elev_dropdown
    with pc_output:
        pc_output.clear_output(True)
        
        # Run ATL03 Subsetter
        print(f'SlideRule processing request... initiated\r', end="")
        perf_start = time.perf_counter()
        atl03_rsps = runATL03Subsetter()
        perf_duration = time.perf_counter() - perf_start
        print(f'SlideRule processing request... completed in {perf_duration:.3f} seconds; returned {atl03_rsps.shape[0]} records                                   ')

        # Create Plots
        if atl03_rsps.shape[0] > 0 and atl06_rsps.shape[0] > 0:
            fig,ax = plt.subplots(num=None, figsize=(10, 8))
            fig.set_facecolor('white')
            fig.canvas.header_visible = False
            ax.set_title("Photon Cloud")
            ax.set_xlabel('UTC')
            ax.set_ylabel('height (m)')
            SRwidgets.plot(atl06_rsps, ax=ax, kind='scatter',
                atl03=atl03_rsps, cmap=SRwidgets.colormap,
                classification=SRwidgets.plot_classification.value,
                segments=(elev_dropdown.value == 'enabled'),
                legend=True, legend_frameon=True)
            # draw and show plot
            plt.show()
            plt.draw()
        
# create button to display geodataframe        
pc_button.on_click(on_pc_clicked)

# click handler for individual photons
def click_handler(feature):
    if "properties" in feature:
        if "rgt" in feature["properties"] and "cycle" in feature["properties"] and "gt" in feature["properties"]:
            SRwidgets.rgt.value = str(feature["properties"]["rgt"])
            SRwidgets.cycle.value = str(feature["properties"]["cycle"])
            SRwidgets.ground_track.value = gt2str[feature["properties"]["gt"]]

# install click handler callback
m.add_selected_callback(click_handler)

In [ ]:
# elevation plot drop down
elev_dropdown = widgets.Dropdown(
    options = ["enabled", "disabled"],
    value = "enabled",
    description = "ATL06-SR",
    disabled = False,
)

display(SRwidgets.rgt)
display(SRwidgets.cycle)
display(SRwidgets.ground_track)
display(SRwidgets.plot_classification)
display(elev_dropdown)
display(pc_button)
display(pc_output)